In [12]:
import numpy as np
import pandas as pd
from decision_tree import ID3 as Tree
from sklearn.model_selection import train_test_split

In [13]:
data = pd.read_csv("iris.csv").dropna()
data

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [14]:
categorical_feats = list(data.columns)
for c in categorical_feats:
    data[c] = data[c].astype('category')
determiner = categorical_feats[-1]
features = categorical_feats[:-1]
print(data.dtypes)
print("Determiner Class: ", determiner)

sepallength    category
sepalwidth     category
petallength    category
petalwidth     category
class          category
dtype: object
Determiner Class:  class


In [15]:
# Split data into features (X) and target (y)
X = data.iloc[:, :-1].to_numpy()  # Select all rows, all columns except the last one
y = data.iloc[:, -1].to_numpy()   # Select all rows, only the last column



# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print((len(X_train), len(y_train)), (len(X_test), len(y_test)))

# for i in range(len(features)):
#     print("Feature: ", features[i], end="\n")
#     print("Gain info: ", tree.info_gain(X, i, y), end="\n")

(120, 120) (30, 30)


In [16]:
tree = Tree(max_depth=3)
tree.fit(X_train, y_train)

In [17]:
def print_tree(node, feature_names, depth=0):
    indent = "  " * depth
    if node.value is not None:
        print(f"{indent}Leaf: Class {node.value}")
    else:
        feature_name = feature_names[node.feature_index]
        print(f"{indent}Feature '{feature_name}':")
        for param, child in node.children:
            print(f"{indent}  If '{feature_name}' == {param}:")
            print_tree(child, feature_names, depth + 2)

In [18]:
print_tree(tree.root, features)

Feature 'petallength':
  If 'petallength' == 1.0:
    Leaf: Class Iris-setosa
  If 'petallength' == 1.2:
    Leaf: Class Iris-setosa
  If 'petallength' == 1.3:
    Leaf: Class Iris-setosa
  If 'petallength' == 1.4:
    Leaf: Class Iris-setosa
  If 'petallength' == 1.5:
    Leaf: Class Iris-setosa
  If 'petallength' == 1.6:
    Leaf: Class Iris-setosa
  If 'petallength' == 1.7:
    Leaf: Class Iris-setosa
  If 'petallength' == 1.9:
    Leaf: Class Iris-setosa
  If 'petallength' == 3.0:
    Leaf: Class Iris-versicolor
  If 'petallength' == 3.3:
    Leaf: Class Iris-versicolor
  If 'petallength' == 3.5:
    Leaf: Class Iris-versicolor
  If 'petallength' == 3.6:
    Leaf: Class Iris-versicolor
  If 'petallength' == 3.8:
    Leaf: Class Iris-versicolor
  If 'petallength' == 3.9:
    Leaf: Class Iris-versicolor
  If 'petallength' == 4.0:
    Leaf: Class Iris-versicolor
  If 'petallength' == 4.1:
    Leaf: Class Iris-versicolor
  If 'petallength' == 4.2:
    Leaf: Class Iris-versicolor
  If '

In [19]:
y_pred = tree.predict(X_test)

In [20]:
def accuracy(y_true, y_pred):
    correct_predictions = np.sum(y_true == y_pred)
    total_predictions = len(y_true)
    return correct_predictions / total_predictions

In [21]:
accuracy(y_test, y_pred)

0.8666666666666667

In [22]:
y_pred

['Iris-setosa',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor']